In [2]:
from Environment.custom_env import CustomEnvironment
from pettingzoo.test import parallel_api_test
from pettingzoo.mpe import simple_adversary_v3
from Models.ActorCritic import ActorCritic
import json

In [65]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [66]:
f = open('Models/model_config.json')
model_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [67]:
ac = ActorCritic(model_variables)

In [68]:
env = CustomEnvironment(env_config=env_variables)
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [69]:
parallel_api_test(env, num_cycles=1000)

k:\ANACONDA\envs\RL\Lib\site-packages\pettingzoo\test\parallel_test.py:90: UserWarning: Agent was given info but was dead last turn
  warnings.warn(f"Agent was given {k} but was dead last turn")


agent0 consumed pellet7
agent2 consumed pellet8
agent2 consumed pellet90
agent1 died at time 99
agent0 died at time 120
agent2 died at time 143


k:\ANACONDA\envs\RL\Lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given observation
  warnings.warn(f"Live agent was not given {k}")
k:\ANACONDA\envs\RL\Lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given info
  warnings.warn(f"Live agent was not given {k}")
k:\ANACONDA\envs\RL\Lib\site-packages\pettingzoo\test\parallel_test.py:90: UserWarning: Agent was given terminated but was dead last turn
  warnings.warn(f"Agent was given {k} but was dead last turn")


agent0 consumed pellet24
agent2 consumed pellet4
agent1 consumed pellet40
agent0 died at time 120
agent2 died at time 120
agent1 died at time 121
Passed Parallel API test


In [61]:
observations, infos = env.reset()
while env.agents:
    # print(env.timestep)
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    
    observations, rewards, terminations, truncations, infos = env.step(actions)

# env.close()

agent1 consumed pellet56
agent0 died at time 99
agent2 died at time 99
agent1 died at time 121
